In [1]:

import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')

if gpu_devices:
    tf.config.experimental.set_visible_devices(gpu_devices[1], 'GPU')
    print(gpu_devices[1])
    print('Success')



2024-06-05 14:08:13.621310: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 14:08:13.621425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 14:08:13.873755: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
Success


In [6]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
import os
print(os.listdir('/kaggle/input'))


['bone_marrow_cell_dataset', 'abbreviations.csv']


In [8]:
# Path to your dataset
data_dir = '/kaggle/input/bone_marrow_cell_dataset'

# List to store image paths and labels
image_paths = []
labels = []

# Loop through class directories

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    
    if os.path.isdir(class_dir):
        for sub_dir_name in os.listdir(class_dir):
            sub_dir_path = os.path.join(class_dir, sub_dir_name)
            
            if os.path.isdir(sub_dir_path):
                for img_name in tqdm(os.listdir(sub_dir_path), desc=f"Processing {class_name}/{sub_dir_name}"):
                    img_path = os.path.join(sub_dir_path, img_name)
                    if os.path.isfile(img_path):
                        image_paths.append(img_path)
                        labels.append(class_name)
            else:
                img_path = os.path.join(class_dir, sub_dir_name)
                if os.path.isfile(img_path):
                    image_paths.append(img_path)
                    labels.append(class_name)
    else:
        img_path = class_dir  # For classes without subdirectories
        if os.path.isfile(img_path):
            image_paths.append(img_path)
            labels.append(class_name)

Processing PEB/1001-2000: 100%|██████████| 1000/1000 [00:00<00:00, 3708.09it/s]


In [9]:
# Splitting the data into train, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(image_paths, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Now X_train, y_train, X_val, y_val, X_test, y_test are the organized data for training, validation, and testing
# Combine all labels from train, validation, and test sets
combined_labels = y_train + y_val + y_test

# Print all unique classes
unique_classes = set(combined_labels)
print("Unique classes:", unique_classes)

Unique classes: {'PMO', 'PLM', 'FGC', 'EOS', 'BAS', 'EBO', 'LYT', 'ART', 'KSC', 'BLA', 'NGS', 'MMZ', 'PEB', 'OTH', 'MON', 'NIF', 'LYI', 'NGB', 'MYB', 'ABE', 'HAC'}


In [10]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Calculate the number of classes directly from the labels
num_classes = len(set(combined_labels))

# Convert numerical labels to one-hot encoded vectors
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_val_onehot = to_categorical(y_val_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

num_classes


21

In [11]:
# Change the input shape to match Xception's input shape
base_model = Xception(weights='imagenet', include_top=False, input_shape=(250, 250, 3))

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 124, 124,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 124, 124,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 124, 124,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 122, 122,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 122, 122,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 122, 122,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 122, 122,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 122, 122,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 122, 122,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 122, 122,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 122, 122,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 61, 61,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 61, 61,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 61, 61,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 61, 61,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 61, 61,    │          0 │ add[0][0]       

 Total params: 54,421,565 (207.60 MB)

 Trainable params: 54,367,037 (207.39 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [14]:
# Preprocess images using ImageDataGenerator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  
    rescale=1. / 255
)


batch_size = 32

# Define a custom generator to load and preprocess images on-the-fly
def custom_generator(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    while True:
        indices = np.random.choice(num_samples, size=batch_size, replace=False)
        batch_images = []
        batch_labels = []
        for idx in indices:
            img_path = image_paths[idx]
            label = labels[idx]
            try:
                img = load_img(img_path, target_size=(250, 250))  # Resize images to VGG-16 input size
                img_array = img_to_array(img)
                batch_images.append(img_array)
                batch_labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
        yield np.array(batch_images), to_categorical(label_encoder.transform(batch_labels), num_classes)

In [ ]:

filepath = '/kaggle/working/Xception_weights.hdf5'
callbacks_list = [EarlyStopping(monitor='val_accuracy', patience=10, verbose=1),
                  ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                  ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)]

In [15]:
# Create custom generators for training, validation, and test sets
train_generator = custom_generator(X_train, y_train, batch_size)
val_generator = custom_generator(X_val, y_val, batch_size)
test_generator = custom_generator(X_test, y_test, batch_size)


In [16]:
epochs = 5

# Train the model using the custom generator
model.fit(
    custom_generator(X_train, y_train, batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=custom_generator(X_val, y_val, batch_size),
    validation_steps=len(X_val) // batch_size,
    verbose=1,
#     callbacks=callbacks_list
)

Epoch 1/5


I0000 00:00:1717645567.053104     118 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717645567.121277     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1519/3748 ━━━━━━━━━━━━━━━━━━━━ 19:04 514ms/step - accuracy: 0.4958 - loss: 1.8334Error loading image /kaggle/input/bone_marrow_cell_dataset/MYB/5001-6000/MYB_05527.jpg: broken data stream when reading image file
3748/3748 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - accuracy: 0.6072 - loss: 1.4131

W0000 00:00:1717647554.710658     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3748/3748 ━━━━━━━━━━━━━━━━━━━━ 2262s 581ms/step - accuracy: 0.6072 - loss: 1.4130 - val_accuracy: 0.7788 - val_loss: 0.8250
Epoch 2/5
 994/3748 ━━━━━━━━━━━━━━━━━━━━ 23:30 512ms/step - accuracy: 0.7910 - loss: 0.7241Error loading image /kaggle/input/bone_marrow_cell_dataset/MYB/5001-6000/MYB_05527.jpg: broken data stream when reading image file
2679/3748 ━━━━━━━━━━━━━━━━━━━━ 9:07 512ms/step - accuracy: 0.7881 - loss: 0.7454Error loading image /kaggle/input/bone_marrow_cell_dataset/MYB/5001-6000/MYB_05527.jpg: broken data stream when reading image file
3748/3748 ━━━━━━━━━━━━━━━━━━━━ 2054s 548ms/step - accuracy: 0.7890 - loss: 0.7424 - val_accuracy: 0.7744 - val_loss: 0.7553
Epoch 3/5
 182/3748 ━━━━━━━━━━━━━━━━━━━━ 30:30 513ms/step - accuracy: 0.8067 - loss: 0.6233Error loading image /kaggle/input/bone_marrow_cell_dataset/MYB/5001-6000/MYB_05527.jpg: broken data stream when reading image file
3748/3748 ━━━━━━━━━━━━━━━━━━━━ 2035s 543ms/step - accuracy: 0.8163 - loss: 0.6240 - val_accuracy:

In [23]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Convert y_val to a NumPy array if it isn't already
y_val = np.array(y_val)

# Check the shape of y_val and print a sample
print("Shape of y_val:", y_val.shape)
print("Sample y_val:", y_val[:5])

# Assuming 'model' is your trained model and 'custom_generator' is your data generator

# Generate predictions on the validation set
val_predictions = model.predict(custom_generator(X_val, y_val, batch_size), steps=np.ceil(len(X_val) / batch_size).astype(int))

# Ensure predictions and true labels have the same length
val_predictions = val_predictions[:len(y_val)]

# Determine the format of y_val
if y_val.ndim == 1:
    # If y_val is a list or array of class labels
    val_true_classes = y_val
elif y_val.ndim == 2:
    # If y_val is one-hot encoded
    val_true_classes = np.argmax(y_val, axis=1)
else:
    raise ValueError("Unexpected y_val format")

# Convert predictions to class labels
val_pred_classes = np.argmax(val_predictions, axis=1)

# Ensure predictions and true labels have the same length
val_pred_classes = val_pred_classes[:len(val_true_classes)]

# If val_true_classes are strings, create a label mapping
if isinstance(val_true_classes[0], str):
    unique_labels = sorted(set(val_true_classes))
    label_mapping = {i: label for i, label in enumerate(unique_labels)}
    val_pred_classes = [label_mapping[pred] for pred in val_pred_classes if pred in label_mapping]

# Calculate the confusion matrix
conf_matrix = confusion_matrix(val_true_classes, val_pred_classes)

# Calculate validation accuracy
val_accuracy = accuracy_score(val_true_classes, val_pred_classes)

print(f"Validation Accuracy: {val_accuracy}")

# Plot confusion matrix
plt.figure(figsize=(14, 10))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


Shape of y_val: (25706,)
Sample y_val: ['LYT' 'ART' 'NGS' 'PMO' 'EBO']
804/804 ━━━━━━━━━━━━━━━━━━━━ 109s 136ms/step


ValueError: Found input variables with inconsistent numbers of samples: [25706, 24041]

In [ ]:
model.save('Xception_Model.h5')

In [ ]:
model_final = load_model('Xception_weights.hdf5')

In [39]:
# Evaluate the model on the test set using the custom generator
test_loss, test_accuracy = model_final.evaluate(
    test_generator,
    steps=len(X_test) // batch_size,
    verbose=1,
)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')



NameError: name 'model_final' is not defined

In [ ]:

# Determine the size of the test dataset
test_size = len(X_test)

print("Total number of samples in the test dataset:", test_size)


In [40]:

from math import ceil
from sklearn.metrics import classification_report

# Determine the size of the test dataset
test_size = len(X_test)

# Calculate the number of steps based on batch size
batch_size = 32  # Adjust according to your generator's batch size
test_steps = ceil(test_size / batch_size)

# Initialize empty lists to store true and predicted labels
y_true = []
y_pred = []

# Iterate through the test generator to obtain predictions batch-wise
for i in range(test_steps):
    x_batch, y_batch = next(test_generator)
    y_true.extend(np.argmax(y_batch, axis=1))
    y_pred.extend(np.argmax(model_final.predict(x_batch), axis=1))



NameError: name 'model_final' is not defined

In [41]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Get the list of all possible classes
all_classes = sorted(set(y_true + y_pred + y_train_encoded.tolist()))

# Calculate confusion matrix with all classes
conf_matrix = confusion_matrix(y_true, y_pred, labels=all_classes)

# Plot confusion matrix
plt.figure(figsize=(11.7, 8.27), dpi=200)
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(all_classes))

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, conf_matrix[i, j],
                 ha="center", va="center",
                 color="white" if conf_matrix[i, j] > conf_matrix.max() / 2 else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.xticks(tick_marks, unique_classes, rotation=45)
plt.yticks(tick_marks, unique_classes)
plt.savefig('Xception.png')
plt.show()

# Calculate classification report
class_report = classification_report(y_true, y_pred,target_names=unique_classes)

# Print classification report
print("Classification Report:")
print(class_report)


ValueError: Found input variables with inconsistent numbers of samples: [32, 0]